<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/tfp_ex_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_probability as tfp
import tensorflow as tf


%matplotlib inline
plt.style.use('default')

tfd = tfp.distributions
tfb = tfp.bijectors

z $\sim N(0,1)$ $→$ x $\sim N(5, 0.2)$ 

In [ ]:
N = 1000
X = tfd.Normal(loc=5,scale=0.2).sample(1000)
X = X.numpy().reshape([N,1])
plt.hist(X,30)
X.shape

In [ ]:
a = tf.Variable(0.0) # ortalaması: 0
b = tf.Variable(1.0) # standart sapması: 1

bijector = tfb.Shift(a)(tfb.Scale(b))
dist = tfd.TransformedDistribution(distribution=tfd.Normal(loc=0,scale=1),bijector=bijector)
dist.trainable_variables

In [ ]:
-tf.reduce_mean(dist.log_prob(X)) # Tüm eğitim kümesinin Negatif Log Likelihood(NLL) değeri

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)

for i in range(1000):
    with tf.GradientTape() as tape:
        loss = -tf.reduce_mean(dist.log_prob(X)) 
        gradients = tape.gradient(loss,           
                    dist.trainable_variables)     
    optimizer.apply_gradients(
        zip(gradients, dist.trainable_variables))    
    
    if (i % 100 == 0):
        print(i, "NLL ",loss.numpy(), " shift (a) ", a.numpy(), " scale (b) ", b.numpy())